In [1]:
# !pip install --upgrade langchain
#!pip install --upgrade langchain-groq
# !pip install -U langchain-community
#!pip install python-docx pypandoc
# !pip install python-docx docx2pdf

# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFDirectoryLoader, PyPDFLoader, WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain.agents.agent_toolkits import ZapierToolkit
from docx import Document
import pypandoc
from langchain.utilities.zapier import ZapierNLAWrapper
from langchain.agents import initialize_agent , create_structured_chat_agent, AgentType
import random
from groq import Groq
import streamlit as st
api_keys = ['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','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',
            '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','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']

client = Groq(
    
    api_key = random.choice(api_keys)
)

llm = ChatGroq(groq_api_key = client.api_key,
               model_name = "llama3-70b-8192",temperature=0.0001)

In [3]:
from cryptography.fernet import Fernet
import base64
from dotenv import load_dotenv
import os
# Load environment variables from .env
load_dotenv()
with open('C:\\Users\\pc-user1\\Desktop\\key.txt', 'r') as file:
    k = file.read().strip()[2:-1]
    #k = base64.urlsafe_b64decode(k.encode())


# Decode and decrypt the API key
encoded_encrypted_api_key = os.getenv("ENCRYPTED_API_KEY")
encrypted_api_key = base64.b64decode(encoded_encrypted_api_key)
decipher_suite = Fernet(k)


In [4]:

cc_out = """
RENOVATION QUOTE DOCUMENT
**Project Overview:**
- Project Description: Renovate 150 sq ft area in basement with premium features, modern design, and a budget of $500,000.
- Length of Time for Project: 2 months

**Cost Breakdown:**

1. **Labor Costs:**
   - Total Labor Cost: $250,000

2. **Material Costs:**
   - Total Material Cost: $49,500
   - Itemized Costs:
     + 150 sq ft hardwood flooring: $5,000
     + 100 sq ft carpeting: $2,000
     + 150 sq ft drywall: $1,000
     + 50 sq ft tinted glass: $5,000
     + 100 sq ft mirror: $3,000
     + 50 sq ft modern-style wall panels: $2,000
     + 10 overhead LED lights: $1,000
     + 5 recessed LED lights: $500
     + 2 LED strip lights: $200
     + 10 new electrical outlets: $1,000
     + 2 new light switches: $200
     + 1 new dedicated 20-amp circuit: $1,000
     + 2 new vents: $1,000
     + 1 new return air vent: $500
     + 1 new water supply line: $1,000
     + 1 new drain line: $1,000
     + 100-inch TV: $5,000
     + Large couch: $2,000
     + Pool table: $2,000
     + Foosball table: $1,000
     + Hot tub: $10,000
     + Modern-style coffee table: $1,000
     + Soundproofing material: $1,000
     + Acoustic panels: $1,000
     + Joint compound: $500
     + Primer: $500
     + Painter's tape: $100

**Total Estimated Cost:**
- Total Cost of Labor: $250,000
- Total Cost of Materials: $49,500
- **Grand Total: $299,500**

**Payment Terms:**
- Deposit Required: 50% of the Grand Total ($149,750.00)
- Payment Schedule: Monthly payments of $24,958 for 6 months
- Final Payment Due: Upon completion of the project
"""

In [5]:
import os
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx2pdf import convert
from langchain.tools import BaseTool
import re

llm.groq_api_key = random.choice(api_keys)

def add_section(doc, text, font_name='Arial', font_size=12, bold=False, italic=False, alignment='left', color=None):
    # Check if the text should be a bullet point
    if text.startswith("- ") or text.startswith("+ "):
        paragraph = doc.add_paragraph(style='ListBullet')
        text = text[2:]  # Remove the bullet point indicator
    else:
        paragraph = doc.add_paragraph()
    
    run = paragraph.add_run(text)
    run.font.name = font_name
    run.font.size = Pt(font_size)
    run.bold = bold
    run.italic = italic
    if color:
        run.font.color.rgb = RGBColor(*color)
    
    if alignment == 'center':
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    elif alignment == 'right':
        paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    else:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT


def write_file(title, sections, append=False):
    documents_path = os.path.join(os.getcwd(), 'documents')
    os.makedirs(documents_path, exist_ok=True)
    docx_path = os.path.join(documents_path, f'{title}.docx')
    pdf_path = os.path.join(documents_path, f'{title}.pdf')
    
    if append and os.path.exists(docx_path):
        doc = Document(docx_path)
    else:
        doc = Document()

    for section in sections:
        add_section(
            doc,
            section["content"],
            font_name=section.get("font_name", "Arial"),
            font_size=section.get("font_size", 12),
            bold=section.get("bold", False),
            italic=section.get("italic", False),
            alignment=section.get("alignment", "left"),
            color=section.get("color", None)
        )
    
    doc.save(docx_path)
    convert(docx_path, pdf_path)

    return f"Document written and converted to pdf. No further action needed."


class WriteFileTool(BaseTool):
    name = "write_file"
    description = "Writes content into a .docx file with specified formatting and converts it to a .pdf file.\
        If you encounter errors, use this tool ONLY and input everything properly and try again.\
            IMPORTANT! - DO NOT CALL TOOL MULTIPLE TIMES IF FIRST TIME WORKED ALREADY!"
    
    def _run(self, title, sections, append=False):
        self.process_sections_in_batches(title, sections, append)

    def process_sections_in_batches(self, title, sections, append, batch_size=1):
        for i in range(0, len(sections), batch_size):
            batch = sections[i:i+batch_size]
            result = write_file(title, batch, append=(i > 0))
        return ("STOP RIGHT THERE! THAT IS ENOUGH! NO MORE ! YOU ARE DONE!")


def parse_formatted_response(response):
    title_match = re.search(r'"title":\s*"([^"]+)"', response)
    title = title_match.group(1) if title_match else "Untitled Document"
    
    sections_matches = re.findall(r'\{([^}]+)\}', response)
    sections = []
    for match in sections_matches:
        section = {}
        content_match = re.search(r'"content":\s*"([^"]+)"', match)
        font_name_match = re.search(r'"font_name":\s*"([^"]+)"', match)
        font_size_match = re.search(r'"font_size":\s*(\d+)', match)
        bold_match = re.search(r'"bold":\s*(true|false)', match)
        italic_match = re.search(r'"italic":\s*(true|false)', match)
        alignment_match = re.search(r'"alignment":\s*"([^"]+)"', match)
        color_match = re.search(r'"color":\s*\[(\d+),\s*(\d+),\s*(\d+)\]', match)
        
        section["content"] = content_match.group(1) if content_match else ""
        section["font_name"] = font_name_match.group(1) if font_name_match else "Arial"
        section["font_size"] = int(font_size_match.group(1)) if font_size_match else 12
        section["bold"] = bold_match.group(1) == "true" if bold_match else False
        section["italic"] = italic_match.group(1) == "true" if italic_match else False
        section["alignment"] = alignment_match.group(1) if alignment_match else "left"
        if color_match:
            section["color"] = (int(color_match.group(1)), int(color_match.group(2)), int(color_match.group(3)))
        
        sections.append(section)
    
    append_match = re.search(r'"append":\s*(true|false)', response)
    append = append_match.group(1) == "true" if append_match else False
    
    return title, sections, append


chat_completion = client.chat.completions.create(
    messages=[
        {
            "role":"system",
            "content":"You are a professional formatter. BE ELABORATE."
        },
        {
            "role":"user",
            "content":"please synthesize ALL of this stuff into sentence format. Start the paragraph with '@' character. put the itemized costs in parentheses beside the item: " + cc_out,
        }
    ],
    model = "llama3-70b-8192",
)

cc_out2 = (chat_completion.choices[0].message.content)
idx = cc_out2.find('@')
cc_out2 = cc_out2[idx+1:].strip()
print('ccount2 old\n',cc_out2)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """You are a text formatting expert. 
            You are an expert also at determining what numerous text formatting parameters should be.
            
                        You must always output in a structured format as follows:
                        {
                            "title": "The Great Renovation",
                            "sections": [
                                {
                                    "content": "<specific text content>",
                                    "font_name": "<font family>",
                                    "font_size": <integer specifying font size>,
                                    "bold": <true or false>,
                                    "italic": <true or false>,
                                    "alignment": "<right, left, justify or center>"
                                    "color": [<integer for red>, <integer for green>, <integer for blue>]
                                },
                                {
                                    "content": "<another specific text content>",
                                    "font_name": "<another font family>",
                                    "font_size": <another integer specifying font size>,
                                    "bold": <true or false>,
                                    "italic": <true or false>,
                                    "alignment": "<right, left, justify or center>"
                                    "color": [<integer for red>, <integer for green>, <integer for blue>]
                                }
                                // ... more sections as needed
                            ],
                            "append": <true or false> (this indicates appending to document or not)
                        }
                        Ensure the output is well-formed and valid."""
        },
        {
            "role": "user",
            "content": "TITLE OF DOCUMENT IS 'The Great Renovation'. Include titles, headings, format in a nice readable and coherent way, NOT a big block of text.  \
                Specify the formatting for this text. Please make it a \
                    PROFESSIONALLY formatted text WITH black blue colors and times new roman font PLEASE. ALL SECTIONS MUST BE INCLUDED EVEN IF THERE ARE A LOT : " + cc_out2
        }
    ],
    model="llama3-70b-8192",
)

cc_out2 = chat_completion.choices[0].message.content

print('CCOUT2\n',cc_out2)

# title, sections, append = parse_formatted_response(cc_out2)

# write_file_tool._run(title, sections, append)


c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ccount2 old
 For a premium basement renovation project, spanning 150 sq ft, with a modern design, and a budget of $500,000, the total estimated cost is $299,500, broken down into labor costs of $250,000 and material costs of $49,500, comprising (150 sq ft hardwood flooring: $5,000), (100 sq ft carpeting: $2,000), (150 sq ft drywall: $1,000), (50 sq ft tinted glass: $5,000), (100 sq ft mirror: $3,000), (50 sq ft modern-style wall panels: $2,000), (10 overhead LED lights: $1,000), (5 recessed LED lights: $500), (2 LED strip lights: $200), (10 new electrical outlets: $1,000), (2 new light switches: $200), (1 new dedicated 20-amp circuit: $1,000), (2 new vents: $1,000), (1 new return air vent: $500), (1 new water supply line: $1,000), (1 new drain line: $1,000), (100-inch TV: $5,000), (large couch: $2,000), (pool table: $2,000), (foosball table: $1,000), (hot tub: $10,000), (modern-style coffee table: $1,000), (soundproofing material: $1,000), (acoustic panels: $1,000), (joint compound: $5

In [6]:
# from langchain.tools import BaseTool
# import os
# import smtplib
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText
# from google.oauth2 import service_account
# from googleapiclient.discovery import build
# from googleapiclient.http import MediaFileUpload
# import googleapiclient
# import google.auth.exceptions

# class GoogleDriveUploadTool(BaseTool):
#     name = "GoogleDriveUploadTool"
#     description = ("Uploads a PDF to Google Drive and sets permissions for a specific user. "
#                    "Please set the 'rename' parameter to None if the user does not want to rename the file before uploading "
#                    "to Google Drive.")

#     def _run(self, file_path: str, user_email: str, rename: str) -> str:
#         credentials_path = "C:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json"

#         # Authenticate and create the service
#         try:
#             credentials = service_account.Credentials.from_service_account_file(credentials_path)
#             service = build('drive', 'v3', credentials=credentials)
#             print("Successfully authenticated and created the service.")
#         except google.auth.exceptions.GoogleAuthError as e:
#             return f"Authentication error: {e}"

#         # File details
#         if rename is not None:
#             file_metadata = {'name': rename}
#         else:
#             file_metadata = {'name': os.path.basename(file_path)}

#         if not os.path.exists(file_path):
#             return f"File not found: {file_path}"
#         else:
#             media = MediaFileUpload(file_path, mimetype='application/pdf')
#             try:
#                 # Upload file
#                 file = service.files().create(body=file_metadata, media_body=media, fields='id, name, parents').execute()
#                 file_id = file.get('id')
#                 print(f"File ID: {file_id}")

#                 # Verify the upload by fetching the file details
#                 file_info = service.files().get(fileId=file_id, fields='id, name, parents').execute()
#                 print("File details:")
#                 print(file_info)

#                 # Sharing settings
#                 permission = {
#                     'type': 'user',
#                     'role': 'writer',  # Set to 'reader' if read-only access is needed
#                     'emailAddress': user_email
#                 }

#                 # Grant permission to the specific user
#                 try:
#                     service.permissions().create(
#                         fileId=file_id,
#                         body=permission,
#                         fields='id'
#                     ).execute()
#                     print(f"Granted {permission['role']} access to {user_email}")
                    
#                     # Send a separate email notification
#                     self.send_email_notification(user_email, file_info)

#                 except googleapiclient.errors.HttpError as e:
#                     print(f"Error setting permission: {e}")
#                     return f"Error setting permission: {e}"

#                 return f"File uploaded and permissions set successfully. File ID: {file_id}"

#             except Exception as e:
#                 return f"Error during file upload or conversion: {e}"

#     def send_email_notification(self, user_email, file_info):
#         from_email = "your-email@example.com"
#         from_password = "your-email-password"
#         subject = "File Shared with You on Google Drive"
#         body = f"""
#         Hi,

#         A file has been shared with you on Google Drive. Here are the details:

#         File Name: {file_info['name']}
#         File ID: {file_info['id']}
#         Folder: {file_info.get('parents', ['None'])[0]}

#         You can access the file directly in your Google Drive.

#         Best regards,
#         Your Name
#         """

#         # Create the email
#         msg = MIMEMultipart()
#         msg['From'] = from_email
#         msg['To'] = user_email
#         msg['Subject'] = subject
#         msg.attach(MIMEText(body, 'plain'))

#         # Send the email
#         try:
#             server = smtplib.SMTP('smtp.gmail.com', 587)
#             server.starttls()
#             server.login(from_email, from_password)
#             text = msg.as_string()
#             server.sendmail(from_email, user_email, text)
#             server.quit()
#             print(f"Notification email sent to {user_email}.")
#         except Exception as e:
#             print(f"Failed to send email: {e}")

#     def _arun(self, file_path: str, user_email: str, rename: str):
#         raise NotImplementedError("This tool does not support asynchronous operation yet.")

# # Example usage
# # tool = GoogleDriveUploadTool()
# # response = tool._run(file_path='./documents/The Great Renovation.pdf', user_email='specific-user@example.com', rename=None)
# # print(response)


In [24]:
from langchain.tools import BaseTool
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import googleapiclient.errors
import google.auth.exceptions
import googleapiclient.discovery

class GoogleDriveUploadTool(BaseTool):
    name = "GoogleDriveUploadTool"
    description = ("Uploads a PDF to Google Drive using a user-specific service account. "
                   "Please set the 'rename' parameter to None if the user does not want to rename the file before uploading "
                   "to Google Drive.")

    def _run(self, file_path: str, user_email: str, rename: str) -> str:
        project_id = 'realestateai-doc-mgr'  # Your Google Cloud Project ID
        service_account_name = user_email.replace('@', '-').replace('.', '-')  # Unique name for the service account
        service_account_display_name = f"Service account for {user_email}"

        # Step 1: Create a service account for the user
        try:
            service_account_info = self.create_service_account(project_id, service_account_name, service_account_display_name)
            print(f"Service account created: {service_account_info['email']}")
        except Exception as e:
            return f"Error creating service account: {e}"

        # Step 2: Assign roles to the service account
        try:
            self.assign_roles_to_service_account(project_id, service_account_info['email'])
            print(f"Roles assigned to service account: {service_account_info['email']}")
        except Exception as e:
            return f"Error assigning roles to service account: {e}"

        # Step 3: Use the service account to upload the file to Google Drive
        try:
            file_id = self.upload_file_to_drive(service_account_info, file_path, rename)
            print(f"File uploaded. File ID: {file_id}")
        except Exception as e:
            return f"Error uploading file: {e}"

       

        return f"File uploaded and notification sent successfully. File ID: {file_id}"

    def create_service_account(self, project_id, account_id, display_name):
        credentials = service_account.Credentials.from_service_account_file("C:\\DEV\\WebdevFolder\\admin_service_account_key.json")
        service = googleapiclient.discovery.build('iam', 'v1', credentials=credentials)
        
        service_account_body = {
            'accountId': account_id,
            'serviceAccount': {
                'displayName': display_name
            }
        }
        
        request = service.projects().serviceAccounts().create(
            name=f'projects/{project_id}',
            body=service_account_body
        )
        response = request.execute()
        return response

    def assign_roles_to_service_account(self, project_id, service_account_email):
        credentials = service_account.Credentials.from_service_account_file("C:\\DEV\\WebdevFolder\\admin_service_account_key.json")
        service = googleapiclient.discovery.build('cloudresourcemanager', 'v1', credentials=credentials)

        policy_body = {
            "policy": {
                "bindings": [
                    {
                        "role": "roles/drive.file",
                        "members": [
                            f"serviceAccount:{service_account_email}"
                        ]
                    }
                ]
            }
        }

        request = service.projects().setIamPolicy(
            resource=project_id,
            body=policy_body
        )
        response = request.execute()
        return response

    def upload_file_to_drive(self, service_account_info, file_path, rename):
        credentials = service_account.Credentials.from_service_account_info(
            service_account_info,
            scopes=['https://www.googleapis.com/auth/drive']
        )
        service = build('drive', 'v3', credentials=credentials)

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")

        file_metadata = {'name': rename if rename else os.path.basename(file_path)}
        media = MediaFileUpload(file_path, mimetype='application/pdf')

        file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        return file.get('id')

    
    def _arun(self, file_path: str, user_email: str, rename: str):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")


In [8]:
zapier = ZapierNLAWrapper(zapier_nla_api_key=decipher_suite.decrypt(encrypted_api_key).decode())
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
import json
from langchain.agents import Tool
my_tools = []
for tool in toolkit.tools:
    my_tools.append(tool)


my_tools.append(
    
    WriteFileTool(),
    
)

my_tools.append(
    
    GoogleDriveUploadTool()
)
llm.groq_api_key = random.choice(api_keys)
agent = initialize_agent(my_tools,llm,handle_parsing_errors=True,agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
for tool in my_tools:
    print(tool.name)
    print(tool.description)
    print("\n\n")

Google Drive: Upload File
A wrapper around Zapier NLA actions. The input to this tool is a natural language instruction, for example "get the latest email from my bank" or "send a slack message to the #general channel". Each tool will have params associated with it that are specified as a list. You MUST take into account the params when creating the instruction. For example, if the params are ['Message_Text', 'Channel'], your instruction should be something like 'send a slack message to the #general channel with the text hello world'. Another example: if the params are ['Calendar', 'Search_Term'], your instruction should be something like 'find the meeting in my personal calendar at 3pm'. Do not make up params, they will be explicitly specified in the tool description. If you do not have enough information to fill in the params, just say 'not enough information provided in the instruction, missing <param>'. If you get a none or null response, STOP EXECUTION, do not try to another tool!

In [10]:
title, sections, append = parse_formatted_response(cc_out2)
llm.groq_api_key = random.choice(api_keys)
result = agent.invoke({"input": f"use ONLY write file tool. INPUT EVERYTHING PROPERLY! --> TITLE: {title}, SECTIONS: {json.dumps(sections)}, APPEND: {append}"})
print(result)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "write_file",
  "action_input": {
    "title": "The Great Renovation",
    "sections": [{"content": "PROJECT SUMMARY", "font_name": "Times New Roman", "font_size": 24, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "For a premium basement renovation project, spanning 150 sq ft, with a modern design, and a budget of $500,000, the total estimated cost is $299,500, broken down into labor costs of $250,000 and material costs of $49,500.", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "justify", "color": [0, 0, 0]}, {"content": "PROJECT BREAKDOWN", "font_name": "Times New Roman", "font_size": 18, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 255]}, {"content": "Labor Costs: $250,000", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content"

100%|██████████| 1/1 [00:01<00:00,  1.03s/it]



Observation: None
Thought:Action:
```
{
  "action": "write_file",
  "action_input": {
    "title": "The Great Renovation",
    "sections": [{"content": "PROJECT SUMMARY", "font_name": "Times New Roman", "font_size": 24, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "For a premium basement renovation project, spanning 150 sq ft, with a modern design, and a budget of $500,000, the total estimated cost is $299,500, broken down into labor costs of $250,000 and material costs of $49,500.", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "justify", "color": [0, 0, 0]}, {"content": "PROJECT BREAKDOWN", "font_name": "Times New Roman", "font_size": 18, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 255]}, {"content": "Labor Costs: $250,000", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Material C

In [25]:
file_path, user_email = "./documents/The Great Renovation.pdf", "sriramnallani35@gmail.com"
llm.groq_api_key = random.choice(api_keys)
result = agent.invoke({"input": f"please use google drive upload tool, rename the file to 'Quote' --> FILE_PATH: {file_path} USER_EMAIL: {user_email}"})
print(result)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "GoogleDriveUploadTool",
  "action_input": {
    "file_path": "./documents/The Great Renovation.pdf",
    "user_email": "sriramnallani35@gmail.com",
    "rename": "Quote"
  }
}
```

Observation: Error creating service account: <HttpError 403 when requesting https://iam.googleapis.com/v1/projects/docmgr-reai/serviceAccounts?alt=json returned "Permission iam.serviceAccounts.create is required to perform this operation on project projects/docmgr-reai.". Details: "Permission iam.serviceAccounts.create is required to perform this operation on project projects/docmgr-reai.">
Thought:Action:
```
{
  "action": "GoogleDriveUploadTool",
  "action_input": {
    "file_path": "./documents/The Great Renovation.pdf",
    "user_email": "sriramnallani35@gmail.com",
    "rename": "Quote"
  }
}
```


Observation: Error creating service account: <HttpError 403 when requesting https://iam.googleapis.com/v1/projects/docmgr-reai/serviceAccount

KeyboardInterrupt: 